In [ ]:
import covid
import numpy as np
from datetime import datetime,timedelta, date
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MultipleLocator, FuncFormatter
from IPython.display import clear_output
from IPython.display import Markdown
from cycler import cycler
import ipywidgets as widgets
%matplotlib inline

In [ ]:
# Config

days_range = (date.today() - date(2020,2,24)).days + 5
order = 8
log_scale = False
tick = FuncFormatter(lambda x, p: format(int(x), ','))

data = covid.data
data_pct = covid.data_pct
updated_at = covid.updated_at

# Implementation

def fmt_plot(aplot, size):
  global log_scale

  aplot.grid(which='major', color='#999999', linestyle='--')
  aplot.grid(which='minor', color='#CCCCCC', linestyle=':')
  aplot.set(xlabel="Giorni dal 24/02/2020")
  aplot.yaxis.set_major_formatter(tick)
  aplot.xaxis.set_major_locator(MultipleLocator(10))
  aplot.xaxis.set_minor_locator(AutoMinorLocator(10))
  if log_scale:
    return aplot
  aplot.yaxis.set_major_locator(MultipleLocator(size))
  aplot.yaxis.set_minor_locator(AutoMinorLocator(5))
  aplot.axvline(x=14, color="orange", linestyle="--")
  aplot.axvline(x=26, color="red", linestyle="--")

  return aplot

def format_row_wise(styler, formatter):
    for row, row_formatter in formatter.items():
        row_num = styler.index.get_loc(row)
        for col_num in range(len(styler.columns)):
            styler._display_funcs[(row_num, col_num)] = row_formatter            
    return styler

def draw_reflines(plt, label=False):
    if (label==True):
        plt.axvline(x=14, color="orange", label="lockdown iniziale", linestyle="--")
        plt.axvline(x=26, color="red", label="lockdown completo", linestyle="--")
        plt.axvline(x=69, color="lightgreen", label="riapertura iniziale", linestyle="--")
        plt.axvline(x=83, color="green", label="riapertura completa", linestyle="--")
        plt.axvline(x=99, color="blue", label="spostamenti tra regioni", linestyle="--")
    else:
        plt.axvline(x=14, color="orange", linestyle="--")
        plt.axvline(x=26, color="red", linestyle="--")
        plt.axvline(x=69, color="lightgreen", linestyle="--")
        plt.axvline(x=83, color="green", linestyle="--")    
        plt.axvline(x=99, color="blue", linestyle="--")

def color_negative(val):    
    color = 'red' if val < 0 else 'black'
    weight = 'bold' if val < 0 else 'normal'
    return 'color: %s; font-weight:%s' % (color, weight)

styles = [
    dict(selector="*", props=[("font-size", "105%")])
]


# Dati Coronavirus Italia

In [ ]:
# Data model:
#   'data', 'stato', 'ricoverati_con_sintomi', 'terapia_intensiva',
#   'totale_ospedalizzati', 'isolamento_domiciliare', 'totale_positivi',
#   'variazione_totale_positivi', 'nuovi_positivi', 'dimessi_guariti',
#   'deceduti', 'totale_casi', 'tamponi', 'note_it', 'note_en'


statistics = covid.statistics
recap = covid.recap

display(Markdown("### *Aggiornamento al " + updated_at + "*"))

formatters = {
    'Mortalità': '{:+.2%}'.format,
    'Critici': '{:+.2%}'.format,
    'Ricoverati': '{:+.2%}'.format,
    'Guariti': '{:+.2%}'.format
}

recap_style = recap.style
recap_style.set_table_styles(styles)
recap_style.format('{:,.0f}'.format)
recap_style.format('{:+,.0f}'.format, subset=['Variazione tot rispetto a ieri'])
recap_style.format('{:+.2%}'.format, subset=['Variazione % rispetto a ieri'])
recap_style.applymap(color_negative)
styler = format_row_wise(recap_style, formatters)

display(recap_style)


## Dati ultimi 7 giorni

In [ ]:
pivot = covid.pivot

formatters = {
    '% mortalita': '{:,.2%}'.format,
    '% intensivi': '{:,.2%}'.format,
    '% ricoverati': '{:,.2%}'.format,
    '% guariti': '{:,.2%}'.format
}

pivot_style = pivot.iloc[:,0:7].style
pivot_style.set_table_styles(styles)
pivot_style.format('{:,.0f}'.format)
styler = format_row_wise(pivot_style, formatters)

display(styler)

## Grafici andamenti 


In [ ]:
plt.rcParams['figure.figsize'] = [16, 5]
plt.rcParams['axes.prop_cycle'] = cycler(color='byrgmgk')

#create tabs
out1 = widgets.Output()
out2 = widgets.Output()
tab_nest = widgets.Tab(children = [out1, out2])
tab_nest.set_title(0, 'Lineari')
tab_nest.set_title(1, 'Logaritmici')

def plot_func(columns):    
    global log_scale
    global data
    return data.loc[:,columns] \
            .plot(kind='line',
                subplots=True,
                sharex=True,
                layout=(1,3),
                grid=True,
                legend=True,
                logy=log_scale)

def update_legend(plot, manual=None):
    handles, labels = plot.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plot.legend(by_label.values(), by_label.keys())

def draw_plots(): 
    avg_days = 7    
    plt.rcParams['figure.figsize'] = [16, 5]    
    
    plots = plot_func(['totale_positivi','variazione_totale_positivi','terapia_intensiva'])
    
    plt_casi = fmt_plot(plots[0][0], 20000)
    draw_reflines(plt_casi, True)
    moving_avg = data.variazione_totale_positivi.rolling(window=avg_days).mean()
    update_legend(plt_casi)
    
    plt_positivi = fmt_plot(plots[0][1], 1000)    
    plt_positivi.plot(moving_avg, color='green', linestyle='-.', label='media mobile')
    draw_reflines(plt_positivi)
    update_legend(plt_positivi)
        
    plt_intensivi = fmt_plot(plots[0][2], 1000)
    draw_reflines(plt_intensivi)    
    update_legend(plt_intensivi)
    
    plt.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')
    plots = plot_func(['totale_ospedalizzati','deceduti','dimessi_guariti'])
    plt_ricoverati = fmt_plot(plots[0][0], 10000)
    draw_reflines(plt_ricoverati)
    update_legend(plt_ricoverati)

    plt_deceduti = fmt_plot(plots[0][1], 5000)
    draw_reflines(plt_deceduti)
    update_legend(plt_deceduti)
    
    plt_dimessi = fmt_plot(plots[0][2], 20000)
    draw_reflines(plt_dimessi)
    update_legend(plt_dimessi)
    
    plt.rcParams['axes.prop_cycle'] = cycler(color='yyrrmgk')
    plots = plot_func(['nuovi_decessi','nuovi_positivi','tamponi'])

    plt_nuovi_decessi = fmt_plot(plots[0][0], 200)
    moving_avg = data.nuovi_decessi.rolling(window=avg_days).mean()
    plt_nuovi_decessi.plot(moving_avg, color='blue', label='media mobile', linestyle='-.')
    draw_reflines(plt_nuovi_decessi)    
    update_legend(plt_nuovi_decessi)

    
    plt_nuovi = fmt_plot(plots[0][1], 1000)
    moving_avg = data.nuovi_positivi.rolling(window=avg_days).mean()
    plt_nuovi.plot(moving_avg, color='green', label='media mobile', linestyle='-.')
    draw_reflines(plt_nuovi)
    update_legend(plt_nuovi)
    
    plt_tamponi = fmt_plot(plots[0][2], 500000)
    draw_reflines(plt_tamponi)
    update_legend(plt_tamponi)
                      
    plt.show()

log_scale = False    
with out1:        
    draw_plots()
log_scale = True
with out2:
    plt.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')
    draw_plots()
    
display(tab_nest)


## Variazioni percentuali giorno per giorno

In [ ]:
tick = FuncFormatter(lambda x, p: format(x, '.0%'))
xtick = FuncFormatter(lambda x, p: format(x, '.0f'))
plt.rcParams['figure.figsize'] = [16, 5]
plt.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

def plot_func_pct(columns):
    global log_scale
    plot_pct = data_pct.loc[:,columns] \
        .plot(kind='bar',
              x='data',  
              subplots=True,
              sharex=True,
              layout=(1,3),
              grid=True,
              legend=False)
    plt_fmt = fmt_plot(plot_pct[0][0], .5)
    plt_fmt.yaxis.set_major_formatter(tick)
    plt_fmt.xaxis.set_major_formatter(xtick)
    draw_reflines(plt_fmt, True)
    update_legend(plt_fmt)
    
    plt_fmt = fmt_plot(plot_pct[0][1], .5)
    draw_reflines(plt_fmt)
    update_legend(plt_fmt)
    
    plt_fmt.yaxis.set_major_formatter(tick)
    plt_fmt.xaxis.set_major_formatter(xtick)
    plt_fmt = fmt_plot(plot_pct[0][2], .5)
    draw_reflines(plt_fmt)
    update_legend(plt_fmt)
    
    plt_fmt.yaxis.set_major_formatter(tick)
    plt_fmt.xaxis.set_major_formatter(xtick)    
    
    return plot_pct

plot_pct = plot_func_pct(['data','totale_positivi','totale_casi','terapia_intensiva'])
plot_pct = plot_func_pct(['data','totale_ospedalizzati','deceduti','dimessi_guariti'])
plot_pct[0][2].set_ylim([0,2])
plot_pct = plot_func_pct(['data','nuovi_decessi','nuovi_positivi','tamponi'])


In [ ]:
x = data.index
# y = data.totale_casi
# def exponential_model(x,a,b,c):
#     return a * np.exp(-b * x) + c
# transformer = FunctionTransformer(np.log, validate=True)

# Exponential fit regression
# x_reshaped = x.values.reshape(-1,1)
# y_reshaped = y.values.reshape(-1,1)
# y_trans = transformer.fit_transform(y_reshaped)
pred_x = np.linspace(10, days_range, days_range)
# pred_x_reshape = pred_x.reshape(-1,1)
# model = LinearRegression().fit(x_reshaped, y_trans)     
# y_fit = model.predict(pred_x_reshape)


## Regressione polinomiale

In [ ]:
tick = FuncFormatter(lambda x, p: format(int(x), ','))
plt.rcParams['figure.figsize'] = [16, 10]
# plt.semilogy()

# Regression analysis
def regression(order):
    global poly, poly1, poly2, poly3
    coefficients = np.polyfit(x.values, data.totale_casi.values, order)
    poly = np.poly1d(coefficients)
    coefficients1 = np.polyfit(x.values, data.totale_positivi.values, order)
    poly1 = np.poly1d(coefficients1)
    coefficients2 = np.polyfit(x.values, data.totale_ospedalizzati.values, order)
    poly2 = np.poly1d(coefficients2)
    coefficients3 = np.polyfit(x.values, data.terapia_intensiva.values, order)
    poly3 = np.poly1d(coefficients3)


output2 = widgets.Output()
def draw_predictions():
    clear_output()
    display(days_slider)
    display(order_slider)
    regression(order)
    fmt_plot(plt.gca(), 20000)
    plt.ylabel("Numero totale")
    plt.ylim(1, 150000)
    pred_x = np.linspace(10, days_range, days_range)
    #plt_scatter = plt.scatter(x,data.totale_casi,label="Totale casi", color="red")
    plt_scatter1 = plt.scatter(x,data.totale_positivi,label="Totale positivi", color="blue")
    plt_scatter2 = plt.scatter(x,data.totale_ospedalizzati, label="Ospedalizzati", color="green")
    #plt_scatter3 = plt.scatter(x,data.terapia_intensiva, label="Terapia intensiva", color="orange")
    # plt_exp = plt.plot(pred_x, np.exp(y_fit), label="Regressione esponenziale")
    #plt_poly = plt.plot(pred_x, poly(pred_x), color="red")
    plt_poly1 = plt.plot(pred_x, poly1(pred_x), color="blue")
    plt_poly2 = plt.plot(pred_x, poly2(pred_x), color="green")
    #plt_poly3 = plt.plot(pred_x, poly3(pred_x), color="orange")
    plt.ylim(0,data.totale_positivi.max().item() * 1.1)
    draw_reflines(plt, True)    
    
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())

days_slider = widgets.BoundedIntText(
    min = data.data.size,
    max = 180,
    step = 1,
    description = 'Giorni',
    continuous_update = False,
    value = days_range
)

order_slider = widgets.BoundedIntText(
    min = 2,
    max = 10,
    step = 1,
    description = 'Grado',
    continuous_update = False,
    value = order
)

def days_handler(obj):
    global days_range
    if (obj.new != obj.old):    
        days_range = obj.new
        draw_predictions()        
days_slider.observe(days_handler,'value')

def order_handler(obj):
    global order
    if (obj.new != obj.old):    
        order = obj.new
        draw_predictions()        
order_slider.observe(order_handler,'value')

draw_predictions()
display(output2)


## Analisi predittiva variazione nuovi positivi

In [ ]:
from fbprophet import Prophet

delta_positivi = data[['data', 'variazione_totale_positivi']][20:]
delta_positivi = delta_positivi.rename(columns={'data':'ds', 'variazione_totale_positivi':'y'})
# remove outliers
delta_positivi = delta_positivi.drop(delta_positivi[delta_positivi.y <= -2500].index)
delta_positivi = delta_positivi.drop(delta_positivi[delta_positivi.y >= 4500].index)
# model fit
model = Prophet(daily_seasonality=True, yearly_seasonality=False)
model.fit(delta_positivi)
future_days = model.make_future_dataframe(periods=60, freq='D')
# forecasting
forecast = model.predict(future_days)
fig = model.plot(forecast, ylabel='Numero nuovi positivi', xlabel='Giorni', figsize=[16,10])
axes = fig.gca()
axes.grid(which='major', color='#999999', linestyle='--')
axes.grid(which='minor', color='#CCCCCC', linestyle=':')
axes.xaxis.set_major_locator(MultipleLocator(30))
axes.xaxis.set_minor_locator(AutoMinorLocator(10))
axes.yaxis.set_major_formatter(tick)
axes.yaxis.set_major_locator(MultipleLocator(1000))
axes.yaxis.set_minor_locator(AutoMinorLocator(5))


*fonte*: [Dati DPC](https://github.com/pcm-dpc/COVID-19/blob/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv)